In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Store csv created in part one into a DataFrame
file = pd.read_csv("cities.csv")
file.dropna()
file.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [7]:
# Heatmap of humidity
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

In [10]:
# Set parameters to search for a hotel
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Iterate through 
# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -18.1326021,
                    "lng": 13.882402
                },
                "viewport": {
                    "northeast": {
                        "lat": -18.13097267010728,
                        "lng": 13.88388602989272
                    },
                    "southwest": {
                        "lat": -18.13367232989272,
                        "lng": 13.88118637010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "baa79eb1237e8b4d196fc6c2ea4f75a18ac9e9e2",
            "name": "Hotel Pension Le Manoir",
            "opening_hours": {
                "open_now": false
            },
            "photos": [
                {
                    "height": 2448,
                    "html_attributions": [
              

{
    "html_attributions": [],
    "next_page_token": "CrQCIwEAADgOkPui27OhJJLDUXQTGZNg16935cMX7lq3QfFtk93_qPvoPHr_b5RiUTR1UsAROFRbNEcxx7LuruabvfJTLSW-PJAGemQowMLlz744GvnuNCyTTDizmOpbdvYr_7rl2n5UdYki42ITiYbgQT4oyQRf4EQuzCgLGj8C_hyuUmgHjAgKwBl9b3SJATpd-E4viS438UL6mFxXuUYG2ll_jbkiuAneXSxXqBXUl8DRRPXKhMard7qEmkrF95m5FxtBt5AOzxBTYvVblmaTfm8qXD4uUR6NB8GGxPnH6Q2H7ppqcX17e8eLmvsj0OMK-fiCjPclV9TWMoNwwalEGb21B_j0ROaj6IOEwarye7xLInHT684L090SoGqGpYIymCUeM0xDWjjOVXk-vLGs_Ys0rvISEIZ52nlfEbp170JA5fS1FMAaFArQ-A7HbShq-KVuKuZlKm4eDXpz",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -20.536313,
                    "lng": -45.606485
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.53466567010728,
                        "lng": -45.60516722010728
                    },
                    "southwest": {
                        "lat": -20.53736532989272,
                    

{
    "html_attributions": [],
    "next_page_token": "CrQCIwEAAG3I3xdkQtAbp5TUANWYM-gUJJP7lKPWIbtDvLMIJMPLLxs4vHQNBaxR4n5hT5DLJbX62BAge-rFhP4JT-aPPwAaWsOtFZzehdqe4WNiJtzzkkznOhtvBgS34mHM-ts_wdUKbqFLzWXl70qjO98B66F-wZE9t5SmsoaxLJNNcYcsko1wJhfEcczO79gzSTs5UFwuyNPQdBb8iay7OuC_DqSwTh2Umnitfxt-K9fhMVMf9r2TH4pf_XV021W_0-L0Af2g7LCV5LGNyG0KwuGqZQGXhDUASev5L-9kh13qNbd0lCB50RHG2JMvCT7pPne6_21E7kmyLtADclapKYpnk4UFcX5jZnmPNeqUDN-83STo3H3z3LYh1DKyv56reGdRl5Z_nmlgvQUH3LJNQ_c5EXESEACmSQFh4uRNQ4N87ND92xcaFLpqmwDPjbSU5NN20xLy6v4qBRs8",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -29.761924,
                    "lng": -50.019275
                },
                "viewport": {
                    "northeast": {
                        "lat": -29.76054577010728,
                        "lng": -50.01799577010728
                    },
                    "southwest": {
                        "lat": -29.76324542989272,
                    

{
    "html_attributions": [],
    "next_page_token": "CrQCIwEAAB9Iqoy6GRjcY4be7wl0LMdhth4M5mwGxLIKXWoeddhpiocpL8rgKg8hHd_ZhFb2QH0Y0yHs3DyzvDB-48qaR8H_lkk0k98JNTEo1lbC7rvTu-Emk85mWpzyFLSjTb001urr5tT7MnUs-_JPDtPsYHHZzupM-i2OTfU5XHMhC5T5fpVRlsOxKYi37PoQKKbtWi49hticJD3wXkdxAs5Ks_z7HcMKOfRm8LvxkNdV5g3XRErmn1H5S7SudFHH0B-hx4k33ObRE_d2vQEEP9acys4Ky-3qy6zTENGs9V1YTILvWFCuPblIqqq9LI2Qn2-OiXbY7tSVZEkss0SaC_Yk5JO-cgPhMvh6VIMB7wwKLro7Qmgjc7Alt6TlkCb2eolIi-v045V4FADqEB3x4Ydp2DQSEBV6aSiatTEWjX7EayB0rScaFKyLNbYx66qONUTJ33ScA8gqCb2j",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -20.2600637,
                    "lng": 44.3104238
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.25917815,
                        "lng": 44.31223074999999
                    },
                    "southwest": {
                        "lat": -20.26272035,
                        "lng": 4

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 15.0040551,
                    "lng": 74.032279
                },
                "viewport": {
                    "northeast": {
                        "lat": 15.00535382989272,
                        "lng": 74.03354577989272
                    },
                    "southwest": {
                        "lat": 15.00265417010728,
                        "lng": 74.03084612010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "7481f733341b77b9956a3badfe6ac2b68e6e8122",
            "name": "Oceanic Hotel",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/113116447481431456013/photos\">Ocea

{
    "html_attributions": [],
    "next_page_token": "CrQCIwEAAJEUAgb4CM__ul7AmBMmTi2M96RmIB9GCzQHNjdZPjwzsFpSfy2x5ZnNKUll5HRz0IM8EDs72XaUu9FmuUc2PHqi3xi-k0ygvLGvIjGQd7OTgiHnybb6n8ZSK47XO4Px5pAm8JhXhxy9Dt9c9pF23dwYytXI9wEkXGq-FARnH__itQiRx48I8pbTocK2zPbVH7BHSUZ7KzhgbspxdhXGCs-4EATdUlS78AqSdzYnluGs_RWQvWiu_OhGsqPwAmcE3V-GQg8PEOZ9FDRx9JJbDAEoa_CsQYAXrrrtVJ9bSQXPIM0qyZr27V6vrwnbSd4FNMtpTLVLXDdqHdiLUP1YyEDaYS7mtpmBKSLCJ8Jr04T7DkNzu5LWo7PlgKmVXoa9ICfcsYFpuHck_2hjMjjJ1RUSELAHLe7yOPWCf7Ta5yMetPsaFL7gCFzyeDWMz2gZI5dR1OFMptsT",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 35.1782726,
                    "lng": -2.9221445
                },
                "viewport": {
                    "northeast": {
                        "lat": 35.17954257989272,
                        "lng": -2.920968020107277
                    },
                    "southwest": {
                        "lat": 35.17684292010728,
                      

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 22.9164309,
                    "lng": 96.4563258
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.9177379,
                        "lng": 96.45768732989272
                    },
                    "southwest": {
                        "lat": 22.9125099,
                        "lng": 96.45498767010729
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f63499003fa624669e9c027aebacd2f2a4a8d167",
            "name": "Golden Butterfly Hotel",
            "photos": [
                {
                    "height": 1108,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/112290800190781055439/photos\">\u53ef\u

{
    "html_attributions": [],
    "next_page_token": "CrQCIwEAAJyxrUquACTM_MzBvOVsAKqFT4SJ0WceQR3ccV4U-0f5baOSKxljX9V-KLRS7o9RJppF-swxaGphZFBl52HvUkS_t8U3kxKssh2DRQzFGCXB3b6YipeLaL45Bsbq2aqRWiFnB48MlEvTufS1aFwl_drydJzDMzyYaQILWktKBzdvKn_N69mSSwQyVFUDjC73toz38jSJoLdiwQxjuj736tl7IpUmAb1eS49BOTyo1iDCUYwgncRJ20b1XwLeT2eNIjRJquQmus37opfnsC6jhJEsu3O4GNjj5P8sAZ8XnQKO9oXAtU_E3DX_5wzn8UoDZJZWx4GD4ZshSGnsvJjpLdUHl4R6mKJPTYHyC1-YzEkDsfDIEeENFIbutPNC0TR3Vd0hbWOPVNvlPvxlrb3KKMkSEE4EzLEjA6NKb4xjbR80bKUaFPvS_I7NQxR7gbZBQuXv47hvRFXo",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 32.8802013,
                    "lng": 59.2162932
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.88151887989271,
                        "lng": 59.21771767989272
                    },
                    "southwest": {
                        "lat": 32.87881922010727,
                       

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Hotel Pension Le Manoir
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Furnaspark Resort
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Rediadri
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Kimony Resort Hotel
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Oceanic Hotel
5,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Mercure Rif Nador
6,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Golden Butterfly Hotel
7,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,هتل پرويز


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))